In [54]:
import math
from random import randint

import matplotlib.pyplot as plt

In [12]:
def printB(b):
    s = ''
    for i in range(8):
        if i%2 == 0:
                s += "   "
        for j in range(4):
            p = str(b[j+4*i])
            one = len(p) == 1
            s += p+' '*(6-len(p))
        s+='\n'
    print(s)

In [13]:
def move(f,t,b, crown = True):
    p = b[f]
    b[f] = 0
    b[t] = p
    if crown:
        if(t < 4 and p == -1):
            b[t] = -3
        elif(t >=28 and p == 1):
            b[t] = 3

In [14]:
def calcEat(pos, board):
    diag = diagonal[pos]
    peg = board[pos]
    for i in range(4):
        correctDir = (abs(peg) == 3) or (peg == -1 and i<2) or (peg == 1 and i>=2)
        if(correctDir and len(diag[i])>1 and board[diag[i][0]]*peg < 0 and board[diag[i][1]] == 0):
            return 1
    return 0


In [15]:
def positions(b, aiTurn):
    return [i for i in range(32) if (b[i] > 0 if aiTurn else b[i] < 0)]

In [19]:
def canEat(b, aiTurn):
    return [i for i in positions(b, aiTurn) if calcEat(i, b)]

In [20]:
def allMoves(b, aiTurn):
    moves = canEat(b, aiTurn)
    if len(moves) == 0:
        x = [singleMove(i,b) for i in positions(b, aiTurn)]
    else:
        x = [multiMove([i],b) for i in moves]
    #x = [getMoves(i, b, len(moves) > 0) for i in positions(b)]
    return [z for y in x for z in y]

In [21]:
def singleMove(pos, board):
    diag = diagonal[pos];
    moves = []
  
    peg = board[pos];
    for i in range(4):
        correctDir = (abs(peg) == 3) or (peg == -1 and i<2) or (peg == 1 and i>=2); 
        if(correctDir):
            if len(diag[i])>0 and board[diag[i][0]] == 0:
                moves.append([pos, diag[i][0]]);
            #elif(len(diag[i])>1 and board[diag[i][0]]*peg < 0 and board[diag[i][1]] == 0):
                #moves.append([pos, diag[i][0], diag[i][1]]);
    return moves;

In [22]:
def multiMove(path, board):
    pos = path[-1]
    p = board[pos]
    res = []
    diag = diagonal[pos]
    for i in range(4):
        correctDir = (abs(p)==3 or (p == -1 and i<2) or (p == 1 and i>=2))
        if correctDir and len(diag[i])>1 and board[diag[i][0]]*p < 0 and board[diag[i][1]] == 0:
            #print(board)
            b = board.copy()
            move(pos, diag[i][0], b, False)
            move(diag[i][0], diag[i][1], b, False) 
            t = multiMove(path+[diag[i][0], diag[i][1]], b)
            if len(t) == 0:
                res.append(path+[diag[i][0], diag[i][1]])
            else:
                res += t
    return res

In [249]:
def runWrapper(b, d, ai = True):
    best = -1000 if ai else 1000
    path = []
    for m in allMoves(b, ai):
        b2 = b.copy()
        for i in range(len(m) - 1):
            move(m[i],m[i+1],b2)
        s = run(b2, d - 1) if ai else run2(b2, d - 1)
        if (s > best) if ai else (s < best):
            best = s
            path = [m]
        elif s == best:
            path.append(m)
    if len(path) == 0:
        return "GAMEOVER", best
    #print(path)
    return path[randint(0, len(path)-1)], best


In [105]:
def run(b, d):
    if d == 0:
        #printB(b)
        return sum(b)
    aiTurn = d%2 == 0
    miniMax = -1000 if aiTurn else 1000
    for m in allMoves(b, aiTurn):
        b2 = b.copy()
        for i in range(len(m) - 1):
            move(m[i],m[i+1],b2)
        s = run(b2, d-1)
        if s > miniMax if aiTurn else s < miniMax:
            miniMax = s
    return miniMax

In [130]:
def run2(b, d):
    if d == 0:
        #printB(b)
        return sum(b)
    aiTurn = d%2 == 1
    miniMax = -1000 if aiTurn else 1000
    for m in allMoves(b, aiTurn):
        b2 = b.copy()
        for i in range(len(m) - 1):
            move(m[i],m[i+1],b2)
        s = run(b2, d-1)
        if s > miniMax if aiTurn else s < miniMax:
            miniMax = s
    return miniMax

In [251]:
def play(d1, d2, ai = True):
    b = []
    for i in range(32):
        b.append(0)
    for i in range(12):
        b[i] = 1
        b[31-i] = -1
    while(True):
        nxt, score = runWrapper(b, d1 if ai else d2, ai)
        if nxt == 'GAMEOVER':
            print(("BLUE" if ai else "RED") + ' WINS')
            printB(b)
            break
        for i in range(len(nxt)-1):
            move(nxt[i], nxt[i+1], b)
        #print(nxt, score)
        #printB(b)
        ai =  not ai
    return ai

In [252]:
b = []
for i in range(32):
    b.append(0)
for i in range(12):
    b[i] = 1
    b[31-i] = -1

In [352]:
l = []

In [ ]:
for i in range(1000):
    l.append(play(4, 4, False))

RED WINS
   0     0     0     3     
0     0     0     0     
   0     0     0     0     
0     0     0     0     
   0     0     0     0     
3     0     0     0     
   0     0     0     0     
0     0     0     0     

RED WINS
   0     0     0     0     
0     0     0     0     
   0     0     0     1     
1     1     0     0     
   0     0     0     1     
0     1     0     0     
   0     0     0     0     
0     3     0     3     

RED WINS
   0     0     0     1     
0     1     0     0     
   0     0     0     1     
0     0     0     0     
   0     1     0     0     
1     1     0     0     
   0     0     0     0     
0     3     0     0     

RED WINS
   3     0     0     0     
0     0     0     0     
   0     0     0     0     
3     0     0     0     
   0     3     1     0     
0     3     0     0     
   0     0     0     0     
0     0     0     0     

RED WINS
   0     0     0     1     
0     0     0     1     
   0     0     0     0     
0     0     0     1   

In [ ]:
l.count(False)

In [186]:
ai = True
while(True):
    nxt, score = runWrapper(b, 6, ai)
    for i in range(len(nxt)-1):
        move(nxt[i], nxt[i+1], b)
    print(nxt, score)
    printB(b)
    ai =  not ai

[9, 13] 0
   1     1     1     1     
1     1     1     1     
   1     0     1     1     
0     1     0     0     
   0     0     0     0     
-1    -1    -1    -1    
   -1    -1    -1    -1    
-1    -1    -1    -1    

[21, 17] 0
   1     1     1     1     
1     1     1     1     
   1     0     1     1     
0     1     0     0     
   0     -1    0     0     
-1    0     -1    -1    
   -1    -1    -1    -1    
-1    -1    -1    -1    

[10, 15] 0
   1     1     1     1     
1     1     1     1     
   1     0     0     1     
0     1     0     1     
   0     -1    0     0     
-1    0     -1    -1    
   -1    -1    -1    -1    
-1    -1    -1    -1    

[22, 18] 0
   1     1     1     1     
1     1     1     1     
   1     0     0     1     
0     1     0     1     
   0     -1    -1    0     
-1    0     0     -1    
   -1    -1    -1    -1    
-1    -1    -1    -1    

[15, 18, 22] 0
   1     1     1     1     
1     1     1     1     
   1     0     0     1     
0     1  

ValueError: empty range for randrange() (0,0, 0)

In [162]:
printB(b)

hi
   1     1     1     1     
1     1     1     1     
   1     1     1     1     
0     0     0     0     
   0     0     0     0     
-1    -1    -1    -1    
   -1    -1    -1    -1    
-1    -1    -1    -1    



In [161]:
nxt, score = runWrapper(b, 7, False)
nxt, score

[[20, 16], [21, 16], [21, 17], [22, 17], [22, 18], [23, 18], [23, 19]]


([23, 18], 0)

In [132]:
for i in range(len(nxt)-1):
    move(nxt[i], nxt[i+1], b)

In [526]:
printB(b)

   0     0     0     0     
0     1     0     1     
   0     1     0     0     
0     1     -1    0     
   0     0     1     0     
-1    0     0     1     
   0     0     0     -1    
3     0     0     0     



In [530]:
canEat(b)

[]

In [22]:
BEST_MOVE

In [23]:
allMoves(b2)

NameError: name 'b2' is not defined

In [24]:
b2[22] = 0

NameError: name 'b2' is not defined

In [381]:
b2[5]

1

In [383]:
multiMove([14], b2)

[[14, 18, 23, 26, 30], [14, 17, 21, 25, 30], [14, 17, 21, 24, 28]]

In [373]:
b2[28] = 0

In [374]:
printB(b2)

   1     1     1     0     
1     1     1     1     
   0     0     1     1     
1     0     1     1     
   0     -1    -1    -1    
-1    0     -1    0     
   -1    -1    -1    -1    
0     -1    0     -1    



In [313]:
b2 = [1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, -1, -1, -1, -1, 0, -1, 0, -1, -1, -1, -1, -1, -1, 0, -1]

In [34]:
diagonal = []
for i in range(32):
    diagonal.append([])

n = 0

diagonal[n].append([])
diagonal[n].append([])
diagonal[n].append([5, 9, 14, 2, 18, 23, 27])
diagonal[n].append([4])

#1
n = 1

diagonal[n].append([])
diagonal[n].append([])
diagonal[n].append([6, 10, 15, 19])
diagonal[n].append([5, 8, 12])

#2 
n = 2

diagonal[n].append([])
diagonal[n].append([])
diagonal[n].append([7, 11])
diagonal[n].append([6, 9, 13, 16, 20])

#3 
n = 3

diagonal[n].append([])
diagonal[n].append([])
diagonal[n].append([])
diagonal[n].append([7, 10, 14, 17, 21, 24, 28])

#4
n = 4

diagonal[n].append([])
diagonal[n].append([0])
diagonal[n].append([8, 13, 17, 22, 26, 31])
diagonal[n].append([])

#5
n = 5

diagonal[n].append([0])
diagonal[n].append([1])
diagonal[n].append([9, 14, 18, 23, 27])
diagonal[n].append([8, 12])

#6
n = 6

diagonal[n].append([1])
diagonal[n].append([2])
diagonal[n].append([10, 15, 19])
diagonal[n].append([9, 13, 16, 20])

#7
n = 7

diagonal[n].append([2])
diagonal[n].append([3])
diagonal[n].append([11])
diagonal[n].append([10, 14, 17, 21, 24, 28])

#8
n = 8

diagonal[n].append([4])
diagonal[n].append([5, 1])
diagonal[n].append([13, 17, 22, 26, 31])
diagonal[n].append([12])

#9
n = 9

diagonal[n].append([5, 0])
diagonal[n].append([6, 2])
diagonal[n].append([14, 18, 23, 27])
diagonal[n].append([13, 16, 20])

#10
n = 10

diagonal[n].append([6, 1])
diagonal[n].append([7, 3])
diagonal[n].append([15, 19])
diagonal[n].append([14, 17, 21, 24, 28])

#11
n = 11

diagonal[n].append([7, 2])
diagonal[n].append([])
diagonal[n].append([])
diagonal[n].append([15, 18, 22, 25, 29])

#12
n = 12

diagonal[n].append([])
diagonal[n].append([8, 5, 1])
diagonal[n].append([16, 21, 25, 30])
diagonal[n].append([])

#13
n = 13

diagonal[n].append([8, 4])
diagonal[n].append([9, 6, 2])
diagonal[n].append([17, 22, 26, 31])
diagonal[n].append([16, 20])

#14
n = 14

diagonal[n].append([9, 5, 0])
diagonal[n].append([10, 7, 3])
diagonal[n].append([18, 23, 27])
diagonal[n].append([17, 21, 24, 28])

#15
n = 15

diagonal[n].append([10, 6, 1])
diagonal[n].append([11])
diagonal[n].append([19])
diagonal[n].append([18, 22, 25, 29])

#16
n = 16

diagonal[n].append([12])
diagonal[n].append([13, 9, 6, 2])
diagonal[n].append([21, 25, 30])
diagonal[n].append([20])

#17
n = 17

diagonal[n].append([13, 8, 4])
diagonal[n].append([14, 10, 7, 3])
diagonal[n].append([22, 26, 31])
diagonal[n].append([21, 24, 28])

#18
n = 18

diagonal[n].append([14, 9, 5, 0])
diagonal[n].append([15, 11])
diagonal[n].append([23, 27])
diagonal[n].append([22, 25, 29])

#19
n = 19

diagonal[n].append([15, 10, 6, 1])
diagonal[n].append([])
diagonal[n].append([])
diagonal[n].append([23, 26, 30])

#20
n = 20

diagonal[n].append([])
diagonal[n].append([16, 13, 9, 6, 2])
diagonal[n].append([24, 29])
diagonal[n].append([])

#21
n = 21

diagonal[n].append([16, 12])
diagonal[n].append([17, 14, 10, 7, 3])
diagonal[n].append([25, 30])
diagonal[n].append([24, 28])

#22
n = 22

diagonal[n].append([17, 13, 8, 4])
diagonal[n].append([18, 15, 11])
diagonal[n].append([26, 31])
diagonal[n].append([25, 29])

#23
n = 23

diagonal[n].append([18, 14, 9, 5, 0])
diagonal[n].append([19])
diagonal[n].append([27])
diagonal[n].append([26, 30])

#24
n = 24

diagonal[n].append([20])
diagonal[n].append([21, 17, 14, 10, 7, 3])
diagonal[n].append([29])
diagonal[n].append([28])

#25
n = 25

diagonal[n].append([21, 16, 12])
diagonal[n].append([22, 18, 15, 11])
diagonal[n].append([30])
diagonal[n].append([29])

#26
n = 26

diagonal[n].append([22, 17, 13, 8, 4])
diagonal[n].append([23, 19])
diagonal[n].append([31])
diagonal[n].append([30])

#27
n = 27

diagonal[n].append([23, 18, 14, 9, 5, 0])
diagonal[n].append([])
diagonal[n].append([])
diagonal[n].append([31])

#28
n = 28

diagonal[n].append([])
diagonal[n].append([24, 21, 17, 14, 10, 7, 3])
diagonal[n].append([])
diagonal[n].append([])

#29
n = 29

diagonal[n].append([24, 20])
diagonal[n].append([25, 22, 18, 15, 11])
diagonal[n].append([])
diagonal[n].append([])

#30
n = 30

diagonal[n].append([25, 21, 16, 12])
diagonal[n].append([26, 23, 19])
diagonal[n].append([])
diagonal[n].append([])

#31
n = 31

diagonal[n].append([26, 22, 17, 13, 8, 4])
diagonal[n].append([27])
diagonal[n].append([])
diagonal[n].append([])